# Bi-LSTM Model Retrain

## Load Up-to-date Data from DB

In [7]:
import os

import pandas as pd

from os.path import join
from dotenv import load_dotenv
from sqlalchemy import create_engine, text

In [34]:
ROOT = '/home/sdc/DR_DemandForecast/DemandForecast/'

In [30]:
def db_conn():
    # Load the environment variables from the .env file
    env_file = join(ROOT, 'src', '.env')
    load_dotenv(env_file)

    # Get the values of host, user, pswd, db, and schema from the environment variables
    DBHOST = os.getenv('host')
    DBUSER = os.getenv('user')
    DBPSWD = os.getenv('pswd')
    DBNAME = os.getenv('db')
    SCHEMA = 'public'

    # Use the values as needed
    engine = create_engine(f"postgresql://{DBUSER}:{DBPSWD}@{DBHOST}/{DBNAME}?options=-csearch_path%3D{SCHEMA}", echo=False)
    conn = engine.connect()
    
    return conn

In [27]:
def get_dpr(
    limit: int=None
):
    select_query = """SELECT "Date", "Period", "Demand", "TCL", "TransmissionLoss", "Solar"
    FROM public."RealTimeDPR"
    ORDER BY "Date" DESC, "Period" DESC
"""
    if limit:
        select_query += f"    LIMIT {limit}\n;"
    else:
        select_query += ";"
        
    # print(select_query)
    
    dpr = pd.read_sql(select_query, conn)
    
    dpr.fillna(0, inplace=True)
    
    dpr["TotalDemand"] = dpr["Demand"] + dpr["TCL"] + dpr["TransmissionLoss"] + dpr["Solar"]
    
    return dpr

## Load and Retrain Model

In [31]:
import tensorflow as tf
import keras as tk

import glob

In [38]:
if __name__ == "__main__":
    conn = db_conn()
    dpr = get_dpr(limit=336)
    

In [41]:
dpr.tail(5)

,Date,Period,Demand,TCL,TransmissionLoss,Solar,TotalDemand
331,2024-04-22,47,6595.757,0.0,0.0,0.0,6595.757
332,2024-04-22,46,6752.356,0.0,0.0,0.0,6752.356
333,2024-04-22,45,6896.889,0.0,0.0,0.0,6896.889
334,2024-04-22,44,7062.196,0.0,0.0,0.0,7062.196
335,2024-04-22,43,7196.396,0.0,0.0,0.0,7196.396


In [ ]:
# Get a list of all model files in the directory
newest = max(glob.glob(os.path.join(ROOT, 'model', '*/')), key=os.path.getmtime)
model_files = glob.glob(os.path.join(newest, "*.keras"))

# Sort the list of model files by modification time (most recent first)
model_files.sort(key=os.path.getmtime, reverse=True)

# Select the most recent model file
most_recent_model_file = model_files[0]

# Load the selected model
model = tk.models.load_model(most_recent_model_file, )



In [ ]:
conn.close()